In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import matplotlib.dates as mdates
from time import time
import scipy.io
from matplotlib import gridspec

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'
sns.set(style='whitegrid', palette='deep', font_scale=1.2)
# this is for setting the precision to 4, so the data is better distributed on screen
# and the maximum number of columns to display
pd.set_option('max_columns',40, 'precision',4)

In [3]:
# defining some constants
Type=['Network Coordinator', 'Radio - 2.4 GHz','Acquisition - Temperature',
      'Acquisition - Current / Voltage', 'Power - Solar Panel', 'Power - AC/DC Input']

Modules={"00.57.FE.04":'Net-Coordinator',
         "00.57.FE.0E":'Radio-2.4 GHz',
         "00.57.FE.0F":'Radio-2.4 GHz',
         "00.57.FE.06":'Radio-2.4 GHz',
         "00.57.FE.09":'Radio-2.4 GHz',
         "00.57.FE.01":'Radio-2.4 GHz',
         "00.57.FE.05":'Radio-2.4 GHz',
         "00.57.FE.03":'Radio-2.4 GHz',
         "29.E5.5A.24":'Acq-Tempe',
         "A7.CB.0A.C0":'Acq-Current/Volt',
         "34.B2.9F.A9":'P-Solar Panel',
         "01.E9.39.32":'Acq-Current/Volt',
         "A4.0D.82.38":'P-AC/DC Input',
         "9F.8D.AC.91":'Acq-Tempe',
         "50.39.E2.80":'P-Solar Panel'}

column_names=['Timestamp','Module','Type','Temp_Mod', 'VBus',
              'PT100(0)', 'PT100(1)', 'LVL_Dim(1)', 'V_MPPT', 
              'V_Panel','LVL_Drain(1)','VBat', 'V_Supp','Temp_Oil',
              'Temp_gab','V_MPPT_TE','V_Panel_TE']

columns_radio = ['Timestamp','Module','Type', 'Transmitter', 'N1', 'P_N1(dbm)',
                 'N2', 'P_N2(dbm)', 'N3', 'P_N3(dbm)',
                 'N4','P_N4(dbm)', 'N5', 'P_N5(dbm)',
                 'N6', 'P_N6(dbm)', 'N7','P_N7(dbm)']

columns_radio2 = ['Timestamp','Module','Type', 'Receiver', 'Tx1', 'P_Tx1(dbm)',
                 'Tx2', 'P_Tx2(dbm)', 'Tx3', 'P_Tx3(dbm)',
                 'Tx4','P_Tx4(dbm)', 'Tx5', 'P_Tx5(dbm)',
                 'Tx6', 'P_Tx6(dbm)', 'Tx7','P_Tx7(dbm)']

In [4]:
# Loading the data
df = pd.read_csv('../../Features.csv', usecols=column_names, parse_dates=['Timestamp'], infer_datetime_format=True, index_col='Timestamp')

# Add date features. This allows to see the date by day, month, week, etc.
from helper_functions import add_date_features

df = add_date_features(df)
df.head()

,Module,Type,Temp_Mod,VBus,PT100(0),PT100(1),LVL_Dim(1),V_MPPT,V_Panel,LVL_Drain(1),VBat,V_Supp,Temp_Oil,Temp_gab,V_MPPT_TE,V_Panel_TE,hour,day,month,week,weekday,daylight,weekend
Timestamp,,,,,,,,,,,,,,,,,,,,,,,
2017-10-01 00:00:05,00.57.FE.04,Net-Coordinator,25.3,4.736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,10,39,6,0,1
2017-10-01 00:00:42,00.57.FE.04,Net-Coordinator,25.3,4.743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,10,39,6,0,1
2017-10-01 00:01:13,00.57.FE.04,Net-Coordinator,25.3,4.743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,10,39,6,0,1
2017-10-01 00:01:42,00.57.FE.04,Net-Coordinator,25.3,4.743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,10,39,6,0,1
2017-10-01 00:02:05,00.57.FE.04,Net-Coordinator,25.3,4.743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,10,39,6,0,1


In [5]:
# Loading the data
df_powers = pd.read_csv('../../Modules_Powers_2.csv', usecols=columns_radio2, parse_dates=['Timestamp'], 
                 dtype={'P_Tx1(dbm)':float, 'P_Tx2(dbm)':float, 'P_Tx3(dbm)':float,
                        'P_Tx4(dbm)':float, 'P_Tx5(dbm)':float, 'P_Tx6(dbm)':float,
                       'Tx1':str, 'Tx2':str, 'Tx3':str, 'Tx4':str, 'Tx5':str, 'Tx6':str,
                       'Tx7':str, 'P_Tx7(dbm)':float})

In [6]:
from rssi import get_radios, get_receivers

RADIOS = get_radios(df)
RECEIVERS = get_receivers(df_powers)
print('Radios \n',RADIOS)
print('Receivers \n',RECEIVERS)

Radios 
 ['00.57.FE.04' '00.57.FE.0E' '00.57.FE.0F' '00.57.FE.06' '00.57.FE.09'
 '00.57.FE.01' '00.57.FE.03' '00.57.FE.05']
Receivers 
 ['0x0057FE04' '0x0057FE0E' '0x0057FE0F' '0x0057FE06' '0x0057FE09'
 '0x0057FE01' '0x0057FE03' '0x0057FE05']


In [7]:
POWERS = ['P_Tx1(dbm)', 'P_Tx2(dbm)', 'P_Tx3(dbm)', 'P_Tx4(dbm)', 'P_Tx5(dbm)', 'P_Tx6(dbm)', 'P_Tx7(dbm)']
#df_powers.set_index('Timestamp', drop=True, inplace=True)
df_powers.head()

,Timestamp,Module,Type,Receiver,Tx1,P_Tx1(dbm),Tx2,P_Tx2(dbm),Tx3,P_Tx3(dbm),Tx4,P_Tx4(dbm),Tx5,P_Tx5(dbm),Tx6,P_Tx6(dbm),Tx7,P_Tx7(dbm)
0,2017-10-01 00:00:05,00.57.FE.04,Net-Coordinator,0x0057FE04,0x0057FE09,-72.0,0x0057FE01,-75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-10-01 00:00:42,00.57.FE.04,Net-Coordinator,0x0057FE04,0x0057FE09,-72.0,0x0057FE01,-75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-10-01 00:01:13,00.57.FE.04,Net-Coordinator,0x0057FE04,0x0057FE09,-72.0,0x0057FE01,-75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-10-01 00:01:42,00.57.FE.04,Net-Coordinator,0x0057FE04,0x0057FE09,-72.0,0x0057FE01,-75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-10-01 00:02:05,00.57.FE.04,Net-Coordinator,0x0057FE04,0x0057FE09,-72.0,0x0057FE01,-75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
